In [1]:
from dash import Dash, dcc, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np
import re
from functools import reduce
app = Dash(__name__)

In [2]:
df_twc = pd.read_csv('..\Data\Top_Worlds_Companies\companies.csv')
def contains_numeric(string):
    pattern = r'\d'
    match = re.search(pattern, string)
    if match:
        return np.nan
    elif string == 'Ahemedabad':
        return np.nan
    else:
        return string
def string_to_num(amount):
    r = 0
    try:
        if amount[-1]=='k':
            r = 1000*float(amount[:-1])
        else:
            r = float(amount)
        return r
    except ValueError:
        return np.nan
    
df_twc['desc'] = df_twc['Description'].str.split('|').str[0]
df_twc['sector'] = df_twc['desc'].apply(contains_numeric)
df_twc.drop('desc',axis=1, inplace=True)
df_twc['HRF'] = df_twc['Highly_rated_for'].str.split(', ')
df_twc['Avg_salary'] = df_twc['Avg_salary'].apply(string_to_num)
df_twc

,Company_name,Description,Ratings,Highly_rated_for,Critically_rated_for,Total_reviews,Avg_salary,Interviews_taken,Total_jobs_available,Total_benefits,sector,HRF
0,TCS,IT Services & Consulting | 1 Lakh+ Employees |...,3.8,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits",73.1k,856900.0,6.1k,847,11.5k,IT Services & Consulting,"[Job Security, Work Life Balance]"
1,Accenture,IT Services & Consulting | 1 Lakh+ Employees |...,4.0,"Company Culture, Skill Development / Learning,...",NaN,46.4k,584600.0,4.3k,9.9k,7.1k,IT Services & Consulting,"[Company Culture, Skill Development / Learning..."
2,Cognizant,IT Services & Consulting | 1 Lakh+ Employees |...,3.9,Skill Development / Learning,Promotions / Appraisal,41.7k,561500.0,3.6k,460,5.8k,IT Services & Consulting,[Skill Development / Learning]
3,Wipro,IT Services & Consulting | 1 Lakh+ Employees |...,3.8,Job Security,"Promotions / Appraisal, Salary & Benefits",39.2k,427400.0,3.7k,405,5k,IT Services & Consulting,[Job Security]
4,Capgemini,IT Services & Consulting | 1 Lakh+ Employees |...,3.9,"Job Security, Work Life Balance, Skill Develop...","Promotions / Appraisal, Salary & Benefits",34k,414400.0,2.8k,719,4k,IT Services & Consulting,"[Job Security, Work Life Balance, Skill Develo..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Techila Global Services,IT Services & Consulting | 501-1k Employees | ...,3.7,"Work Life Balance, Salary & Benefits, Company ...",NaN,72,454.0,2,26,21,IT Services & Consulting,"[Work Life Balance, Salary & Benefits, Company..."
9996,RxLogix Corporation,Pharma | 201-500 Employees | 14 years old | Pr...,2.6,"Work Life Balance, Work Satisfaction, Company ...",NaN,72,799.0,15,9,13,Pharma,"[Work Life Balance, Work Satisfaction, Company..."
9997,Avians Innovations Technology,Building Material | 51-200 Employees | 17 year...,3.7,"Promotions / Appraisal, Work Satisfaction, Sal...",NaN,72,489.0,3,11,8,Building Material,"[Promotions / Appraisal, Work Satisfaction, Sa..."
9998,ACPL Systems,Law Enforcement & Security | 51-200 Employees ...,3.3,"Promotions / Appraisal, Salary & Benefits, Wor...",NaN,72,520.0,4,1,10,Law Enforcement & Security,"[Promotions / Appraisal, Salary & Benefits, Wo..."


In [3]:
# companies that you might like to work in based off sectors, ratings and positive critiques

@app.callback(
    Output("fig1", "figure"),
    Input("sector-filter", "value"),
    Input("critique-filter", "value")
)
def update_Sect_Ratings(sector, crit):
    # Apply filters to the data
    filtered_df = df_twc
    filtered_df.dropna(subset=['sector','HRF'], inplace=True)
    if sector:
        filtered_df = filtered_df[filtered_df["sector"].isin(sector)]
    if crit:
        filtered_df = filtered_df[filtered_df["HRF"].apply(lambda x: bool(set(x) & set(crit)))]
#     filtered_df = filtered_df[(filtered_df["Runtime"] >= runtime[0]) & (
#         filtered_df["Runtime"] <= runtime[1])]

    # Create the bar chart
    fig = px.bar(filtered_df.sort_values("Avg_salary", ascending=False).head(20),
                 x="Company_name",
                 y="Avg_salary",
                 color="Ratings",
                 hover_data=["Avg_salary"])
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.update_layout(
        font=dict(color='#fff'),
        xaxis_title="Company",
        yaxis_title="Average Salary",
        title="Top companies by rating",
        plot_bgcolor='#333', paper_bgcolor='#333'

    )
    return fig

In [4]:
hrf_ser = df_twc['HRF'].dropna()
union_HRF = reduce(set.union, map(set, hrf_ser))

In [5]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H2(
        children='Companies based off your likings',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

     dbc.Row([
                html.Label("Sector", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="sector-filter",
                    options=[{"label": s, "value": s}
                             for s in df_twc['sector'].dropna().unique()],
                    value=None,
                    multi=True,
                    style={
                        "width": "100%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": "#333",  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
                html.Label("Features", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="critique-filter",
                    options=[{"label": l, "value": l}
                             for l in union_HRF],
                    value=None,
                    multi=True,
                    style={
                        "width": "100%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": "#333",  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
         dcc.Graph(id="fig1")
     ])
])

In [6]:
if __name__ == '__main__':
    app.run(debug=True)